In [1]:
## statisitical natural language processing
## Project Member: Jurat, Aldo

### Data-Preprocessing


In [56]:
## remove stop words from given fake news statement
from spacy.tokenizer import Tokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from spacy import load
from spacy import displacy
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
import pandas as pd
import re
# import csv


In [17]:
## read dataset
df=pd.read_csv('train.tsv',delimiter='\t',encoding='utf-8')
df.columns=['statement_ID','label','statement','subject','speaker','job_title',
           'state_info','pantry_affiliation','barely_true_cnt','false_cnt',
           'half_true_cnt','mostly_true_cnt','pants_on_fire_cnt','context']

df.head()
df.tail()
df.shape[0]
list(df.columns.values)

['statement_ID',
 'label',
 'statement',
 'subject',
 'speaker',
 'job_title',
 'state_info',
 'pantry_affiliation',
 'barely_true_cnt',
 'false_cnt',
 'half_true_cnt',
 'mostly_true_cnt',
 'pants_on_fire_cnt',
 'context']

In [ ]:
## data cleaning
df.statement=df.statement.str.replace("\n","")
df['nExclmas']=df.statement.str.count("!")

## remove all stop words in statement column
stop=stopwords.words('english')
df['statement']=df['statement'].apply(lambda x: "".join([word for word in x.split() if word not in (stop)]))
df['statement'].head()

## helper function to lemmatize the token
def lemmetize_token(sentence):
    lemmati=WordNetLemmatizer()
    lemmatized = [[lemmati.lemmatize(word) for word in word_tokenize(s)] for s in sentence]
    return(list(lemmatized))

lem_sent=[]
for i in range(len(df.index)):
    res=lemmetize_token(df['statement'][i])
    lem_sent.append(res)
    
##
print(lem_sent)

In [30]:
from sklearn.feature_extraction.text import CountVectorizer 
vectorizer = CountVectorizer(ngram_range=(1,4))
ng_builder = vectorizer.build_analyzer()

##
ng_1=ng_builder(df['statement'][1])
print(list(ng_1))

# below code needs to be optimized
all_grams=[]
for i in range(0, len(df.index)):
    sentence=re.sub(r'a-zA-Z',"", df['statement'][i])
    all_grams.append(ng_builder(sentence))

##
res=pd.DataFrame(all_grams)

['hillaryclintonagreesjohnmccain', 'byvotinggivegeorgebushbenefitdoubtiran', 'hillaryclintonagreesjohnmccain byvotinggivegeorgebushbenefitdoubtiran']


10239

In [38]:
## helper function to remove the token that are punctuation or whitespace
# def process_txt(token):
#     res=token.is_punct or token.is_space
#     return res

## helper function to remove punctuation
from string import punctuation
import string


def rm_punct(sentence):
    flushed_punct=set(string.punctuation)
    res=''.join(x for x in sentence if x not in flushed_punct)
    return res

cleaned_sent=[]
for i in range(len(df.index)):
    res=rm_punct(df['statement'][i])
    cleaned_sent.append(res)
    

['WhendeclinecoalstartItstartednaturalgastookstartedbeginPresidentGeorgeWBushsadministration',
 'HillaryClintonagreesJohnMcCainbyvotinggiveGeorgeBushbenefitdoubtIran',
 'Healthcarereformlegislationlikelymandatefreesexchangesurgeries',
 'Theeconomicturnaroundstartedendterm',
 'TheChicagoBearsstartingquarterbackslast10yearstotalnumbertenuredUWfacultyfiredlasttwodecades',
 'JimDunnamliveddistrictrepresentsyearsnow',
 'ImpersonstageworkedactivelylastyearpassingalongRussFeingoldtoughestethicsreformsinceWatergate',
 'Howevertook195millionOregonLotteryfundsPortNewporteventuallylandnewNOAAMarineOperationsCenterPacific',
 'SaysGOPprimaryopponentsGlennGrothmanJoeLeibhamcastcompromisevotecost788millionhigherelectricitycosts',
 'ForfirsttimehistorysharenationalpopularvotemarginsmallerLatinovotemargin',
 'Since2000nearly12millionAmericansslippedmiddleclasspoverty',
 'WhenMittRomneygovernorMassachusettsdidntslowrategrowthgovernmentactuallycutit',
 'Theeconomybled24billionduegovernmentshutdown',
 'Mo

In [44]:
## helper function to get bag of words
from nltk.tokenize import word_tokenize

def tokenize_sent(sentence):
    token=word_tokenize(sentence)
    return(token)

bag_of_words=[]
for line in range(len(df.index)):
#     sentence=re.sub(ur'[0-9]+|\p{P}+', '', line)
    sentence=tokenize_sent(df['statement'][line])
    bag_of_words.append(sentence)
    
##
bag_of_words

[['Whendeclinecoalstart',
  '?',
  'Itstartednaturalgastookstartedbegin',
  '(',
  'PresidentGeorgeW',
  '.',
  ')',
  'Bushsadministration',
  '.'],
 ['HillaryClintonagreesJohnMcCain',
  "''",
  'byvotinggiveGeorgeBushbenefitdoubtIran',
  '.',
  "''"],
 ['Healthcarereformlegislationlikelymandatefreesexchangesurgeries', '.'],
 ['Theeconomicturnaroundstartedendterm', '.'],
 ['TheChicagoBearsstartingquarterbackslast10yearstotalnumbertenured',
  '(',
  'UW',
  ')',
  'facultyfiredlasttwodecades',
  '.'],
 ['JimDunnamliveddistrictrepresentsyearsnow', '.'],
 ["I'mpersonstageworkedactivelylastyearpassing",
  ',',
  'alongRussFeingold',
  ',',
  'toughestethicsreformsinceWatergate',
  '.'],
 ['However',
  ',',
  'took',
  '$',
  '19.5millionOregonLotteryfundsPortNewporteventuallylandnewNOAAMarineOperationsCenter-Pacific',
  '.'],
 ['SaysGOPprimaryopponentsGlennGrothmanJoeLeibhamcastcompromisevotecost',
  '$',
  '788millionhigherelectricitycosts',
  '.'],
 ['Forfirsttimehistory',
  ',',
  'sha

In [ ]:
## helper function to reomve white space in sentence with lower case character
def sent_lower_case(df):
    sent_arr=[]
    for i in range(0, len(df.index)):
        patt=re.sub(r'[^a-zA-Z]','', df['statement'][i])
        sent='.'.join(x.lower() for x in patt.split('.'))
        sent_arr.append(sent)
    return sent_arr

## helper function for depndencing parsing on fake news statement
def dep_par_sents(df):
    nlp=en_core_web_sm.load()
    corpus=sent_lower_case(df)
    dep_pars_res=[]
    for line in range(0, len(df.index)):
        docs=nlp(unicode(corpus[line],"utf-8"))
        dep_pars_res.append(docs)

# to visualize parsing tree
docs=[]
for each in range(0, len(df.index)):
    displacy.serve(df[each], style='dep')